In [ ]:
# 1. 구글 리서치 깃허브 코드 다운로드
!git clone https://github.com/google-research/language.git

# 2. ToTTo 데이터셋 다운로드 및 압축 풀기
!wget https://storage.googleapis.com/totto-public/totto_data.zip
!unzip -q totto_data.zip

# 3. 필요한 라이브러리 설치
import os
os.chdir('/content/language/language/totto') # 작업 폴더로 이동
!pip install -r eval_requirements.txt

Cloning into 'language'...
remote: Enumerating objects: 4053, done.
remote: Counting objects: 100% (440/440), done.
remote: Compressing objects: 100% (209/209), done.
remote: Total 4053 (delta 330), reused 231 (delta 231), pack-reused 3613 (from 4)
Receiving objects: 100% (4053/4053), 6.29 MiB | 21.53 MiB/s, done.
Resolving deltas: 100% (2304/2304), done.
--2026-01-19 14:25:12--  https://storage.googleapis.com/totto-public/totto_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.189.207, 192.178.129.207, 142.251.184.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.189.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187724372 (179M) [application/zip]
Saving to: ‘totto_data.zip’

totto_data.zip      100%[===================>] 179.03M   284MB/s    in 0.6s    

2026-01-19 14:25:13 (284 MB/s) - ‘totto_data.zip’ saved [187724372/187724372]



In [ ]:
!cat /content/language/language/totto/totto_parent_eval.py

# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
r"""Script to compute PARENT metric modified for ToTTo dataset.

https://arxiv.org/abs/1906.01081

Modified from:
https://github.com/google-research/language/tree/master/language/table_text_eval


The <reference_file> and <generation_file> should contain references and
generations, respectively, one per line. The <table_file> should contain the
ground truth tables corresponding to t

In [ ]:
# ==============================================================================
# 1. 라이브러리 설치 및 평가 스크립트 준비
# ==============================================================================
!pip install -q transformers datasets accelerate sentencepiece sacrebleu

# PARENT 평가 스크립트 다운로드 (없을 경우를 대비해)
!git clone https://github.com/google-research/language.git 2>/dev/null
!pip install -r language/language/totto/eval_requirements.txt -q

import os
import json
import pandas as pd
import numpy as np
import torch
import sacrebleu
from ast import literal_eval
from datasets import load_dataset
from transformers import (
    BartTokenizerFast,           # [핵심] 에러 방지용 Fast 토크나이저
    BartForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)

In [ ]:
# ==============================================================================
# 2. 데이터 전처리 (스키마 통일 & 99% Cutoff)
# ==============================================================================
# 파일 경로 (사용자 환경에 맞게 확인 필요)
input_train_file = "/content/totto_data/totto_train_data.jsonl"
filtered_train_file = "/content/totto_data/totto_train_data_filtered.jsonl"
input_dev_file = "/content/totto_data/totto_dev_data.jsonl"
clean_dev_file = "/content/totto_data/totto_dev_data_clean.jsonl"

# 남길 컬럼 정의 (데이터 충돌 방지)
target_columns = [
    "table", "highlighted_cells", "table_page_title",
    "table_section_title", "table_section_text", "sentence_annotations"
]

print("🔄 [1/5] 데이터 정제 및 필터링 시작...")

# 2-1. 학습 데이터 처리
if not os.path.exists(filtered_train_file):
    df_train = pd.read_json(input_train_file, lines=True)

    # 하이라이트 파싱
    def parse_highlight(x):
        try: return literal_eval(x) if isinstance(x, str) else x
        except: return x

    if len(df_train) > 0:
        df_train["highlighted_cells"] = df_train["highlighted_cells"].apply(parse_highlight)

    # 99% Cutoff
    counts = df_train["highlighted_cells"].apply(lambda x: len(x) if isinstance(x, list) else 0)
    cutoff = counts.quantile(0.99)
    df_train = df_train[counts <= cutoff]

    # 컬럼 통일 및 저장
    df_train = df_train[target_columns]
    df_train.to_json(filtered_train_file, orient="records", lines=True)
    print(f"✅ 학습 데이터 저장 완료 (Cutoff: {cutoff})")

# 2-2. 검증 데이터 처리
if not os.path.exists(clean_dev_file):
    df_dev = pd.read_json(input_dev_file, lines=True)
    df_dev = df_dev[target_columns]
    df_dev.to_json(clean_dev_file, orient="records", lines=True)
    print("✅ 검증 데이터 저장 완료")

🔄 [1/5] 데이터 정제 및 필터링 시작...


In [ ]:
# ==============================================================================
# 3. Linearizer (테이블 -> 마크다운 변환기)
# ==============================================================================
class MarkdownLinearizer:
    def __init__(self): pass

    def _extract_subtable(self, table, highlighted_cells):
        if not table or not highlighted_cells: return []

        # 관련 행/열 찾기
        highlighted_set = set(tuple(cell) for cell in highlighted_cells)
        relevant_rows, relevant_cols = set(), set()
        for r, c in highlighted_set:
            relevant_rows.add(r); relevant_cols.add(c)
        if table:
            relevant_rows.add(0) # 헤더 포함
            for c in range(len(table[0])): relevant_cols.add(c)

        subtable = []
        for r in sorted(relevant_rows):
            if r >= len(table): continue
            new_row = []
            for c in sorted(relevant_cols):
                if c < len(table[r]):
                    cell = table[r][c].copy()
                    cell['_is_highlighted'] = (r, c) in highlighted_set
                    new_row.append(cell)
            subtable.append(new_row)
        return subtable

    def _table_to_markdown(self, subtable):
        if not subtable: return ""
        lines = []
        # 헤더 처리
        headers = [c.get("value", "").strip() or " " for c in subtable[0]]
        lines.append("| " + " | ".join(headers) + " |")
        lines.append("| " + " | ".join(["---"] * len(headers)) + " |")

        # 데이터 처리
        for row in subtable[1:]:
            row_vals = []
            for i, cell in enumerate(row):
                val = cell.get("value", "").strip() or " "
                if cell.get("_is_highlighted", False):
                    head = headers[i] if i < len(headers) else ""
                    val = f"<selected> {head} : {val} </selected>" if head else f"<selected> {val} </selected>"
                row_vals.append(val)
            lines.append("| " + " | ".join(row_vals) + " |")
        return "\n".join(lines)

    def linearize_from_dict(self, data):
        parts = ["Based on the highlighted cells, describe the data."] # 기본 Instruction
        # 메타데이터 추가
        for key, prefix in [('table_page_title', 'Page:'), ('table_section_title', 'Section:')]:
            if data.get(key): parts.append(f"**{prefix}** {data[key].strip()}")

        subtable = self._extract_subtable(data.get("table", []), data.get("highlighted_cells", []))
        if subtable:
            parts.append(self._table_to_markdown(subtable))
        return "\n\n".join(parts)

In [ ]:
# ==============================================================================
# 4. 모델 및 데이터셋 준비
# ==============================================================================
print("🔄 [2/6] 모델 및 데이터셋 준비 중...")
tokenizer = BartTokenizerFast.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("microsoft/tapex-base")
tokenizer.add_special_tokens({'additional_special_tokens': ['<selected>', '</selected>']})
model.resize_token_embeddings(len(tokenizer))

linearizer = MarkdownLinearizer()
dataset = load_dataset("json", data_files={"train": filtered_train_file, "validation": clean_dev_file})

TRAIN_SIZE = 50000
print(f"⚠️ 학습 데이터를 {TRAIN_SIZE}개로 제한하여 실험합니다.")
train_dataset = dataset["train"].shuffle(seed=42).select(range(TRAIN_SIZE))
eval_dataset = dataset["validation"].select(range(1000)) # 검증은 속도를 위해 1000개 유지
def preprocess_function(examples):
    inputs, targets = [], []
    for i in range(len(examples['table'])):
        item = {k: examples[k][i] for k in target_columns if k != 'sentence_annotations'}
        inputs.append(linearizer.linearize_from_dict(item))
        targets.append(examples['sentence_annotations'][i][0]['final_sentence'])
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    labels = tokenizer(text_target=targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

print("🔄 [3/6] 토크나이징 진행 (병렬 처리)...")
tokenized_train = train_dataset.map(preprocess_function, batched=True, num_proc=8)
tokenized_eval = eval_dataset.map(preprocess_function, batched=True, num_proc=8)

🔄 [2/6] 모델 및 데이터셋 준비 중...
⚠️ 학습 데이터를 50000개로 제한하여 실험합니다.
🔄 [3/6] 토크나이징 진행 (병렬 처리)...


Map (num_proc=8):   0%|          | 0/50000 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# ==============================================================================
# [진짜_최종_수정] 학습 재개 (OverflowError 원인인 preds의 -100까지 제거)
# ==============================================================================
import numpy as np
import sacrebleu
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

# 1. 메트릭 계산 함수 (여기만 바꾸면 해결됩니다)
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # Pad ID 확보 (없으면 0)
    pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0

    # [🚨 핵심 수정] labels뿐만 아니라 'preds'도 -100을 없애야 합니다!
    # Trainer가 예측값(preds)의 빈 공간도 -100으로 채울 때가 있어서 에러가 났던 것입니다.
    if isinstance(preds, np.ndarray):
        preds = np.where(preds != -100, preds, pad_token_id)  # <--- 이 줄이 추가됨
        preds = preds.tolist()  # 리스트 변환

    if isinstance(labels, np.ndarray):
        labels = np.where(labels != -100, labels, pad_token_id)
        labels = labels.tolist()  # 리스트 변환

    # 이제 preds와 labels 모두 깨끗한 정수 리스트이므로 에러가 날 수 없습니다.
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 빈 문자열 처리
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    if len(decoded_preds) == 0:
        return {"bleu": 0.0}

    result = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels])
    return {"bleu": result.score}

# 2. 학습 설정 (기존과 동일)
training_args = Seq2SeqTrainingArguments(
    output_dir="./tapex_50k_final_fix",
    predict_with_generate=True,
    generation_max_length=128,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    num_train_epochs=3,
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

# 3. 트레이너 다시 만들기
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("🚀 [최종 재시작] 이제 preds의 -100도 제거했으므로 진짜로 멈추지 않습니다.")
trainer.train()

/tmp/ipython-input-2656666554.py:58: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


🚀 [최종 재시작] 이제 preds의 -100도 제거했으므로 진짜로 멈추지 않습니다.


Epoch,Training Loss,Validation Loss,Bleu
1,0.130500,0.206386,36.310435
2,0.148800,0.194237,37.711764
3,0.134900,0.194182,37.820129


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'num_beams': 4, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=4689, training_loss=0.13885534900316274, metrics={'train_runtime': 1872.5779, 'train_samples_per_second': 80.103, 'train_steps_per_second': 2.504, 'total_flos': 9.1460468736e+16, 'train_loss': 0.13885534900316274, 'epoch': 3.0})

In [ ]:
# ==============================================================================
# 6. PARENT 평가 수행
# ==============================================================================
print("\n📊 [5/6] PARENT 평가를 위한 파일 생성 중...")

# 1) table_content.txt 생성
with open("table_content.txt", "w", encoding="utf-8") as f:
    for i in range(len(eval_dataset)):
        t = eval_dataset[i]['table']
        h = eval_dataset[i]['highlighted_cells']
        cells = []
        for r, c in h:
            if r < len(t) and c < len(t[r]):
                val = t[r][c]['value']
                head = t[0][c]['value'] if len(t) > 0 and c < len(t[0]) else ""
                cells.append(f"{head}|||{val}")
        f.write("\t".join(cells) + "\n")

# 2) 예측 생성
predictions = trainer.predict(tokenized_eval)
preds_ids = np.where(predictions.predictions != -100, predictions.predictions, tokenizer.pad_token_id)
decoded_preds = tokenizer.batch_decode(preds_ids, skip_special_tokens=True)

# 3) generation.txt & reference.txt 생성
with open("generation.txt", "w", encoding="utf-8") as fg, open("reference.txt", "w", encoding="utf-8") as fr:
    for pred, ref_ids in zip(decoded_preds, tokenized_eval['labels']):
        ref = tokenizer.decode([x for x in ref_ids if x != -100], skip_special_tokens=True)
        fg.write(pred.strip() + "\n")
        fr.write(ref.strip() + "\n")

print("📉 [6/6] PARENT 스크립트 실행 결과:")
!python language/language/totto/totto_parent_eval.py \
    --generation_path=generation.txt \
    --reference_path=reference.txt \
    --precision_table_path=table_content.txt \
    --recall_table_path=table_content.txt


📊 [5/6] PARENT 평가를 위한 파일 생성 중...


📉 [6/6] PARENT 스크립트 실행 결과:
Evaluated 1000 examples.
Precision = 62.21 Recall = 42.47 F-score = 45.00


In [ ]:
# ==============================================================================
# [결과 확인] 랜덤 샘플 5개 (Input / Prediction / Reference) 출력
# ==============================================================================
import random
import torch

def show_inference_results_v2(dataset, model, tokenizer, num_samples=5):
    print(f"\n🔍 [결과 확인] 총 {len(dataset)}개 중 랜덤 {num_samples}개 샘플 확인\n")
    print("=" * 80)

    # 모델 평가 모드
    model.eval()
    device = model.device

    # 랜덤 인덱스 선택
    indices = random.sample(range(len(dataset)), num_samples)

    for idx in indices:
        item = dataset[idx] # Raw Data (이미 전처리된 상태여야 함)

        # 1. 입력 데이터 준비 (Batch 차원 추가)
        input_ids = torch.tensor([item['input_ids']]).to(device)

        # 2. 모델 생성 (Generate)
        with torch.no_grad():
            outputs = model.generate(
                input_ids,
                max_length=128,
                num_beams=5,
                early_stopping=True
            )

        # 3. 디코딩 (Special Token 제거)
        # (1) 입력 표 (구조 확인용, Special Token 일부 포함 가능)
        input_text = tokenizer.decode(item['input_ids'], skip_special_tokens=True)

        # (2) 정답 (Reference) - label에서 -100 제거 후 디코딩
        ref_ids = [x for x in item['labels'] if x != -100]
        ref_text = tokenizer.decode(ref_ids, skip_special_tokens=True)

        # (3) 예측 (Prediction)
        pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # 4. 출력
        print(f"📌 [Index {idx}]")
        print(f"\n[입력 (Input Table Content)]:\n{input_text[:10000]} ... (생략)") # 너무 길어서 앞부분만 출력
        print("-" * 40)
        print(f"\n[정답 (Reference)]:\n{ref_text}")
        print(f"\n[예측 (Output)]:\n▶ {pred_text}")
        print("=" * 80 + "\n")

# 실행 (검증 데이터셋 'tokenized_eval' 사용)
# 만약 'tokenized_eval'이 없다면 위쪽 코드에서 정의된 변수명을 확인해주세요.
if 'tokenized_eval' in locals():
    show_inference_results_v2(tokenized_eval, model, tokenizer, num_samples=5)
else:
    print("⚠️ 'tokenized_eval' 데이터셋이 정의되지 않았습니다. 학습 코드를 먼저 실행해주세요.")


🔍 [결과 확인] 총 1000개 중 랜덤 5개 샘플 확인

📌 [Index 250]

[입력 (Input Table Content)]:
Based on the highlighted cells, describe the data.

**Page:** 2012 in AFC

**Section:** Events list

| # | Event Title | Date | Arena | Location |
| --- | --- | --- | --- | --- |
| 4 |  Event Title : AFC 3  |  Date : April 14, 2012  |  Arena : Geelong Arena  |  Location : Geelong, Australia  | ... (생략)
----------------------------------------

[정답 (Reference)]:
AFC 3 was an event held on April 14, 2012, at Geelong Arena in Geelong, Australia.

[예측 (Output)]:
▶ AFC 3 was an event held on April 14, 2012 at Geelong Arena in Geelong, Australia.

📌 [Index 228]

[입력 (Input Table Content)]:
Based on the highlighted cells, describe the data.

**Page:** Evan Helmuth

**Section:** Film

| Year | Title | Role | Notes |
| --- | --- | --- | --- |
|  Year : 2012  |  Title : The Devil Inside  |  Role : Father David Keane  |   | ... (생략)
----------------------------------------

[정답 (Reference)]:
Helmuth was known for his rol